# Dikkat Mekanizmaları ve Transformerlar

Tekrarlayan ağların (recurrent networks) en büyük dezavantajlarından biri, bir dizideki tüm kelimelerin sonuca aynı etkiyi yapmasıdır. Bu durum, Adlandırılmış Varlık Tanıma (Named Entity Recognition) ve Makine Çevirisi gibi dizi-dizi görevlerinde standart LSTM kodlayıcı-çözücü modellerinin performansını düşürür. Gerçekte, giriş dizisindeki belirli kelimeler, sıralı çıktılar üzerinde diğerlerinden daha fazla etkiye sahiptir.

Makine çevirisi gibi bir dizi-dizi modelini düşünelim. Bu model, iki tekrarlayan ağ tarafından uygulanır: biri (**kodlayıcı**) giriş dizisini gizli bir duruma sıkıştırır, diğeri (**çözücü**) bu gizli durumu çevrilmiş bir sonuca açar. Bu yaklaşımın sorunu, ağın son durumunun bir cümlenin başını hatırlamakta zorlanmasıdır, bu da uzun cümlelerde modelin kalitesinin düşmesine neden olur.

**Dikkat Mekanizmaları**, her bir giriş vektörünün RNN'nin her bir çıktı tahmini üzerindeki bağlamsal etkisini ağırlıklandırmanın bir yolunu sunar. Bu, giriş RNN'sinin ara durumları ile çıkış RNN'si arasında kısayollar oluşturarak uygulanır. Bu şekilde, $y_t$ çıktı sembolünü üretirken, farklı ağırlık katsayıları $\alpha_{t,i}$ ile tüm giriş gizli durumlarını $h_i$ dikkate alırız.

![Eklendiği bir dikkat katmanına sahip kodlayıcı/çözücü modelini gösteren görsel](../../../../../translated_images/tr/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau ve diğerleri, 2015](https://arxiv.org/pdf/1409.0473.pdf)'teki toplamsal dikkat mekanizmasına sahip kodlayıcı-çözücü modeli, [bu blog yazısından](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) alıntılanmıştır.*

Dikkat matrisi $\{\alpha_{i,j}\}$, belirli giriş kelimelerinin çıktı dizisindeki bir kelimenin oluşturulmasında ne derece etkili olduğunu temsil eder. Aşağıda böyle bir matrisin örneği verilmiştir:

![RNNsearch-50 tarafından bulunan bir hizalamayı gösteren görsel, Bahdanau - arviz.org'dan alınmıştır](../../../../../translated_images/tr/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau ve diğerleri, 2015](https://arxiv.org/pdf/1409.0473.pdf)'ten alınan şekil (Şekil 3)*

Dikkat mekanizmaları, Doğal Dil İşleme'deki mevcut veya mevcut duruma yakın en iyi performansın büyük bir kısmından sorumludur. Ancak dikkat eklemek, model parametrelerinin sayısını büyük ölçüde artırır ve bu da RNN'lerde ölçekleme sorunlarına yol açar. RNN'lerin ölçeklenmesindeki temel bir kısıtlama, modellerin tekrarlayan doğasının eğitimi toplu işleme ve paralelleştirme açısından zorlaştırmasıdır. Bir RNN'de, bir dizinin her bir öğesi sıralı bir şekilde işlenmelidir, bu da kolayca paralelleştirilemeyeceği anlamına gelir.

Dikkat mekanizmalarının benimsenmesi ve bu kısıtlama, bugün BERT'ten OpenGPT3'e kadar bildiğimiz ve kullandığımız en iyi performansa sahip Transformer Modellerinin oluşturulmasına yol açmıştır.

## Transformer Modelleri

Her bir önceki tahminin bağlamını bir sonraki değerlendirme adımına iletmek yerine, **transformer modelleri**, verilen bir metin penceresi içinde bir girişin bağlamını yakalamak için **konumsal kodlamalar** ve **dikkat** kullanır. Aşağıdaki görsel, konumsal kodlamaların dikkatle birlikte bir pencere içinde bağlamı nasıl yakalayabildiğini göstermektedir.

![Transformer modellerinde değerlendirmelerin nasıl yapıldığını gösteren animasyonlu GIF.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Her bir giriş pozisyonu bağımsız olarak her bir çıkış pozisyonuna eşlendiğinden, transformerlar RNN'lere göre daha iyi paralelleştirilebilir. Bu da çok daha büyük ve daha ifade gücü yüksek dil modellerini mümkün kılar. Her bir dikkat başlığı, kelimeler arasındaki farklı ilişkileri öğrenmek için kullanılabilir ve bu da Doğal Dil İşleme görevlerini iyileştirir.

## Basit Bir Transformer Modeli Oluşturma

Keras, yerleşik bir Transformer katmanı içermez, ancak kendi modelimizi oluşturabiliriz. Daha önce olduğu gibi, AG News veri kümesinin metin sınıflandırmasına odaklanacağız, ancak Transformer modellerinin daha zor NLP görevlerinde en iyi sonuçları verdiğini belirtmekte fayda var.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Keras'ta yeni katmanlar `Layer` sınıfını alt sınıf olarak almalı ve `call` metodunu uygulamalıdır. **Konumsal Gömme** katmanı ile başlayalım. [Resmi Keras dokümantasyonundan bazı kodları](https://keras.io/examples/nlp/text_classification_with_transformer/) kullanacağız. Tüm giriş dizilerini `maxlen` uzunluğuna kadar doldurduğumuzu varsayacağız.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Bu katman, iki `Embedding` katmanından oluşur: biri tokenleri gömmek için (daha önce tartıştığımız şekilde), diğeri ise token pozisyonlarını gömmek için. Token pozisyonları, 0'dan `maxlen` değerine kadar doğal sayıların bir dizisi olarak `tf.range` kullanılarak oluşturulur ve ardından gömme katmanından geçirilir. Elde edilen iki gömme vektörü toplanarak, girişin `maxlen`$\times$`embed_dim` şeklinde pozisyonel olarak gömülü bir temsili elde edilir.

Şimdi, transformer bloğunu uygulayalım. Bu, daha önce tanımlanan gömme katmanının çıktısını alacaktır:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Artık, tam bir transformer modelini tanımlamaya hazırız:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Modelleri

**BERT** (Bidirectional Encoder Representations from Transformers), *BERT-base* için 12 katman ve *BERT-large* için 24 katman içeren çok büyük bir çok katmanlı transformer ağıdır. Model, büyük bir metin veri kümesi (Vikipedi + kitaplar) üzerinde denetimsiz eğitim (bir cümledeki maskelenmiş kelimeleri tahmin etme) kullanılarak önce önceden eğitilir. Ön eğitim sırasında model, önemli bir dil anlama seviyesini öğrenir ve bu bilgi, diğer veri kümeleriyle ince ayar yapılarak kullanılabilir. Bu sürece **transfer öğrenimi** denir.

![http://jalammar.github.io/illustrated-bert/ adresinden alınan görsel](../../../../../translated_images/tr/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT, DistilBERT, BigBird, OpenGPT3 ve daha fazlası gibi ince ayar yapılabilen birçok Transformer mimarisi varyasyonu bulunmaktadır.

Şimdi, önceden eğitilmiş bir BERT modelini kullanarak geleneksel sıralama sınıflandırma problemimizi nasıl çözebileceğimize bakalım. [Resmi dokümantasyon](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)'dan fikir ve bazı kodlar ödünç alacağız.

Önceden eğitilmiş modelleri yüklemek için **Tensorflow hub** kullanacağız. İlk olarak, BERT'e özgü vektörleştiriciyi yükleyelim:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Orijinal ağın eğitildiği vektörleştiriciyi kullanmanız önemlidir. Ayrıca, BERT vektörleştirici üç bileşen döndürür:
* `input_word_ids`, giriş cümlesi için bir dizi token numarasıdır
* `input_mask`, dizinin hangi kısmının gerçek girdi içerdiğini ve hangisinin dolgu olduğunu gösterir. Bu, `Masking` katmanı tarafından üretilen maskeye benzer
* `input_type_ids`, dil modelleme görevlerinde kullanılır ve bir dizide iki giriş cümlesi belirtmeye olanak tanır.

Sonrasında, BERT özellik çıkarıcısını oluşturabiliriz:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Yani, BERT katmanı bir dizi faydalı sonuç döndürür:
* `pooled_output`, dizideki tüm tokenların ortalamasının alınmasıyla elde edilen bir sonuçtur. Bunu tüm ağın akıllı bir anlamsal gömüsü olarak düşünebilirsiniz. Önceki modelimizdeki `GlobalAveragePooling1D` katmanının çıktısına eşdeğerdir.
* `sequence_output`, son transformer katmanının çıktısıdır (yukarıdaki modelimizdeki `TransformerBlock` çıktısına karşılık gelir).
* `encoder_outputs`, tüm transformer katmanlarının çıktılarıdır. 4 katmanlı bir BERT modeli yüklediğimiz için (muhtemelen adından tahmin edebileceğiniz gibi, içinde `4_H` içeriyor), 4 tensöre sahiptir. Sonuncusu `sequence_output` ile aynıdır.

Şimdi uçtan uca sınıflandırma modelini tanımlayacağız. Model girişini tanımladığımız ve ardından çıktısını hesaplamak için bir dizi ifade sağladığımız *fonksiyonel model tanımını* kullanacağız. Ayrıca, BERT model ağırlıklarını eğitilemez hale getirecek ve yalnızca son sınıflandırıcıyı eğiteceğiz:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


BERT özellik çıkarıcısı hesaplama açısından oldukça ağır olduğu için eğitilebilir parametre sayısı az olmasına rağmen süreç oldukça yavaş ilerliyor. Görünüşe göre, ya yeterli eğitim yapılmadığından ya da model parametreleri yetersiz olduğundan makul bir doğruluk elde edemedik.

Şimdi BERT ağırlıklarını serbest bırakmayı ve onu da eğitmeyi deneyelim. Bu, çok küçük bir öğrenme oranı gerektirir ve ayrıca **warmup** ile daha dikkatli bir eğitim stratejisi, **AdamW** optimizasyon algoritmasını kullanmayı gerektirir. Optimizatörü oluşturmak için `tf-models-official` paketini kullanacağız:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Eğitim oldukça yavaş ilerliyor - ancak birkaç epoch (5-10) boyunca modeli eğiterek, daha önce kullandığımız yaklaşımlarla karşılaştırıldığında en iyi sonucu alıp alamayacağınızı denemek isteyebilirsiniz.

## Huggingface Transformers Kütüphanesi

Transformer modellerini kullanmanın başka bir çok yaygın (ve biraz daha basit) yolu [HuggingFace paketi](https://github.com/huggingface/)dir. Bu paket, farklı NLP görevleri için basit yapı taşları sağlar. Hem Tensorflow hem de başka bir çok popüler sinir ağı çerçevesi olan PyTorch için kullanılabilir.

> **Not**: Eğer Transformers kütüphanesinin nasıl çalıştığını görmekle ilgilenmiyorsanız, bu not defterinin sonuna geçebilirsiniz. Çünkü yukarıda yaptıklarımızdan önemli ölçüde farklı bir şey görmeyeceksiniz. Daha büyük bir model ve farklı bir kütüphane kullanarak BERT modelini eğitme adımlarını tekrarlayacağız. Bu nedenle süreç oldukça uzun bir eğitim içeriyor, sadece kodu gözden geçirmek isteyebilirsiniz.

Hadi, [Huggingface Transformers](http://huggingface.co) kullanarak problemimizin nasıl çözülebileceğine bir bakalım.


İlk olarak, kullanacağımız modeli seçmemiz gerekiyor. Dahili modellerin yanı sıra, Huggingface'in [çevrimiçi model deposu](https://huggingface.co/models) bulunmaktadır. Bu depoda topluluk tarafından oluşturulmuş birçok önceden eğitilmiş model bulabilirsiniz. Bu modellerin tamamı, yalnızca model adını sağlayarak yüklenip kullanılabilir. Model için gerekli olan tüm ikili dosyalar otomatik olarak indirilecektir.

Bazı durumlarda kendi modellerinizi yüklemeniz gerekebilir. Bu durumda, tokenizer parametreleri, model parametrelerini içeren `config.json` dosyası, ikili ağırlıklar gibi ilgili tüm dosyaları içeren dizini belirtebilirsiniz.

Model adından, hem modeli hem de tokenizer'ı oluşturabiliriz. Hadi bir tokenizer ile başlayalım:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` nesnesi, metni doğrudan kodlamak için kullanılabilecek `encode` işlevini içerir:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Bir diziyi modele uygun bir şekilde kodlamak için tokenizer'ı da kullanabiliriz, yani `token_ids`, `input_mask` alanları vb. içerecek şekilde. Ayrıca `return_tensors='tf'` argümanını sağlayarak Tensorflow tensörleri istediğimizi belirtebiliriz:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Bu durumda, önceden eğitilmiş `bert-base-uncased` adlı BERT modelini kullanacağız. *Uncased* modelin büyük/küçük harf duyarsız olduğunu belirtir.

Modeli eğitirken, giriş olarak ayrıştırılmış bir dizi sağlamamız gerekir ve bu nedenle veri işleme hattını tasarlayacağız. `tokenizer.encode` bir Python fonksiyonu olduğundan, önceki birimde olduğu gibi `py_function` kullanarak çağırma yaklaşımını kullanacağız:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Artık `BertForSequenceClassification` paketini kullanarak gerçek modeli yükleyebiliriz. Bu, modelimizin sınıflandırma için gerekli olan mimariye, son sınıflandırıcı dahil, zaten sahip olmasını sağlar. Son sınıflandırıcının ağırlıklarının başlatılmadığını ve modelin ön eğitim gerektireceğini belirten bir uyarı mesajı göreceksiniz - bu tamamen normaldir, çünkü tam olarak yapmayı planladığımız şey budur!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


`summary()`'den görebileceğiniz gibi, model neredeyse 110 milyon parametre içeriyor! Muhtemelen, nispeten küçük bir veri kümesinde basit bir sınıflandırma görevi yapmak istiyorsak, BERT temel katmanını eğitmek istemeyiz:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Şimdi eğitime başlamaya hazırız!

> **Not**: Tam ölçekli bir BERT modelini eğitmek oldukça zaman alıcı olabilir! Bu yüzden sadece ilk 32 batch için eğitim yapacağız. Bu, model eğitiminin nasıl kurulduğunu göstermek içindir. Eğer tam ölçekli eğitimi denemek isterseniz, `steps_per_epoch` ve `validation_steps` parametrelerini kaldırın ve beklemeye hazırlanın!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Eğer iterasyon sayısını artırır, yeterince beklerseniz ve birkaç epoch boyunca eğitim yaparsanız, BERT sınıflandırmasının en iyi doğruluğu sağladığını görebilirsiniz! Bunun nedeni, BERT'in dilin yapısını zaten oldukça iyi anlaması ve bizim sadece son sınıflandırıcıyı ince ayar yapmamızın gerekmesidir. Ancak, BERT büyük bir model olduğu için tüm eğitim süreci uzun sürer ve ciddi bir hesaplama gücü gerektirir! (GPU ve tercihen birden fazla).

> **Note:** Örneğimizde, en küçük önceden eğitilmiş BERT modellerinden birini kullanıyoruz. Daha büyük modeller, muhtemelen daha iyi sonuçlar verecektir.


## Özet

Bu bölümde, **transformer** tabanlı çok yeni model mimarilerini inceledik. Bunları metin sınıflandırma görevimizde uyguladık, ancak benzer şekilde BERT modelleri varlık çıkarımı, soru yanıtlama ve diğer NLP görevlerinde de kullanılabilir.

Transformer modelleri, NLP alanında güncel en iyi performansı temsil eder ve çoğu durumda özel NLP çözümleri uygularken denemeye başlamanız gereken ilk çözüm olmalıdır. Ancak, bu modülde tartışılan tekrarlayan sinir ağlarının temel prensiplerini anlamak, gelişmiş sinir modelleri oluşturmak istiyorsanız son derece önemlidir.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
